This notebook presents the semi-physical model of the L1 laser.

Author: Francesco Capuano, 2022 S17 summer intern @ ELI-beamlines, Prague

# Motivation
The goal of this project is to maximise second-harmonic efficiency. However, since it is also very much related to the shortest possible pulse shape, we started with developing a strategy to optimise a predefinite set of control parameters so as to minimise the difference between the obtained pulse shape (in the temporal domain) and a target one (which, by default, is the shortest one typically). 

However, since data are really expensive to empirically collect we resorted to model the underlying dynamics of the whole system, also considering that (even if not exhaustive) there is a significant amount of know-how concerned with the considered dynamics available.
This knowledge about the actual physical process is presented in the following Figure. 

![L1_Pump_SemiPhysicalModel](semi-physical_model.png)

# Description

## Stretcher
A signal of the Eletric field represented in the frequency domain, $E(\nu)$, enters the system from the left, where a phase produced by the **Stretcher** is super-imposed to it. The outcome of said superposition is a new frequency representation the very same electric field, in particular:  

$$ y_1(\nu) = E(\nu) \cdot e^{i \varphi_{stretcher}}$$

The phase $\varphi_{stretcher}$ is obtained with a stretcher and it can be modelled using the following equation (which is obtained as a Taylor Expansion of the phase):

$$
\varphi_{stretcher} = \varphi_{stretcher}(GDD, TOD, FOD; \nu, \nu_0) = \frac12 GDD \big(2\pi (\nu - \nu_0)\big)^2 +
            \frac 16 TOD  \big(2\pi (\nu - \nu_0)\big)^3 + \frac 1{24} FOD \big(2\pi(\nu - \nu_0)\big)^4 $$

With $\nu$ being the considered frequencies and $\nu_0$ being the central carrier frequency (corresponding to the average between the half-max frequencies). 

As it is possible to see from the above Figure, even if the control quantities are not exactly $GDD, TOD$ and $FOD$, the actual control quantities are strictly related to these values. In particular, the actual *control* quantities $d_2, d_3$ and $d_4$ are linked to $GDD, TOD$ and $FOD$ via the following system of linear equations: 

$$
\begin{pmatrix}
-\frac{2 \pi c}{\nu_0^2} & 0 & 0 \\
\frac{4 \pi c}{\nu_0^3} & \big(\frac{2 \pi c}{\nu_0^2}\big)^2 & 0 \\
-\frac{12 \pi c}{\nu_0^4} & \frac{-24 (\pi c)^2}{\nu_0^5} & -\big(\frac{2 \pi c}{\nu_0^2}\big)^3
\end{pmatrix}
\cdot
\begin{pmatrix}
GDD \\
TOD \\
FOD
\end{pmatrix} 
= 
\begin{pmatrix}
d_2 \\
d_3 \\
d_4
\end{pmatrix} 
$$

In our implementation of the code, this system is clearly solved using regular forward substitution. 

## DIRA 
Once the controlled phase is super-imposed to the incoming electric field, $y1(\nu)$ is then propagated through the DIRA block, in which a non-linear phase is accumulated by the signal. This non-linear phase can be modelled (in time only) as: 

$$\varphi_{non\ linear}(t; B) \approx \frac{B}{\max(I(t))} I(t) $$

with $I(t)$ being the intensity (in the time domain) and $B$ being the B-integral, a precise measure of non-linearity whose interpretation falls beyond the scopes of this analysis. However, it is important to note how $B$ has to be considered an hyperparameter of the whole system, i.e. a *non-controlled* quantity which is anyway influencing the dynamics of the system themselves. 
Since this non-linear phase is defined in time-only, its superposition to the incoming $y_1(\nu)$ must take place in the time domain, therefore the DIRA-block can actually be modelled by the following steps: 

1. $y_1(t) \leftarrow $ `ifft`$(y_1(\nu))$; $I(t) \leftarrow y_1(t)^2$

2. $\varphi_{non\ linear}(t; B) \leftarrow \frac{B}{\max(I(t))} I(t)$; $B \approx 2$

3. $y_2(t) \leftarrow y_1(t) \cdot e^{i \varphi_{non\ linear}}(t; B)$

4. $y_2(\nu) \leftarrow$ `fft`$(y_2(t))$

With `fft` and `ifft` being, of course, the Fast Fourier Transform and its inverse. 

## Compressor
Once the non-linear phase has been super-imposed, the signal travels through a compressor, that can be modelled as a block super-imposing once again a *linear* phase. Such linear phase is represented by: 

$$
\varphi_{compressor} = \varphi_{compressor}(\alpha_{GDD}, \alpha_{TOD}, \alpha_{FOD}, \nu, \nu_0) = \frac12 \alpha_{GDD} \big(2\pi (\nu - \nu_0)\big)^2 +
            \frac 16 \alpha_{TOD}  \big(2\pi (\nu - \nu_0)\big)^3 + \frac 1{24} \alpha_{FOD} \big(2\pi(\nu - \nu_0)\big)^4 $$
            
As it is possible to see the phase clearly depends on some parameters, namely $(\alpha_{GDD}, \alpha_{TOD}, \alpha_{FOD})$. These should be considered as a parametrization of the whole system, i.e. as hyperparameters and, therefore *non-controlled* quantities, exactly as $B$. 

The super-imposition of the compressor phase can be modelled with the following equation:

$$y_3(\nu) = y_2(\nu) \cdot e^{i \varphi_{compressor}}$$

## FROG
In this first implementation of the model, once $y_3(\nu)$ is available it is represented in the time-domain (mainly leveraging Fourier transform). The time-based representation of the pulse is then compared with the Transform Limited pulse, which is the one obtained super-imposing a zero-phase to the initial spectral electric field. 

Once this is done, conceptually, a Feedback Signal is propagated backwards to adjust the control parameters so as to minimise such a difference. 

In [1]:
# these imports are necessary to import modules from directories one level back in the folder structure
import sys
import os
import inspect
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parentdir = os.path.dirname(currentdir)
sys.path.insert(0, parentdir)

from utils import LaserModel as lm 
data_path = "../data/LLNL_160809_freq.csv"

In [2]:
# read the data
df = pd.read_csv(data_path, header = None)
df.columns = ["Frequency (in THz)", "Wavelength (in nm)", "Intensity", "Phase (rad)", "Phase (cutted) (rad)"] 
frequency, intensity = df.loc[:, "Frequency (in THz)"], df.loc[:, "Intensity"]

In [4]:
lm = lm.LaserModel(frequency = frequency, intensity = intensity, cutoff = (351, 379))

In [ ]:
fig, ax = plt.subplots()
control = np.array([15, 30, 30])
ax.plot(lm.forward_pass(control = control))